<a href="https://colab.research.google.com/github/PaiwandAlAhmady/RoboCup-Work_Object-Segmentation-Model/blob/main/YOLO12_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q ultralytics -U
%pip install -q depthai==3.0.0rc2 -U
%pip install -q depthai-nodes==0.3.0 -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.3/224.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 53.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
# Lädt Trainingsdatensatz von Drive auf lokalen Colab Speicher -> effizienter
!cp -r /content/drive/MyDrive/RoboCup_Trainingsdatensatz /content/

In [ ]:
from ultralytics import YOLO

model_yolo_12_n = YOLO("yolo12n.pt")
model_yolo_12_s = YOLO("yolo12s.pt")
model_yolo_11_n = YOLO("yolo11n-seg.pt")
model_yolo_11_s = YOLO("yolo11s-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import torch
from ultralytics import YOLO

# Zielarchitektur: v12 SEG, frisch initialisiert
m = YOLO("yolo12n-seg.yaml")
sd_target = m.model.state_dict()

def load_ckpt(path):
    # Fix für PyTorch 2.6: explizit weights_only=False
    ckpt = torch.load(path, map_location="cpu", weights_only=False)
    # typische Ultralytics-Container auflösen
    for k in ("model", "ema", "state_dict"):
        if k in ckpt:
            obj = ckpt[k]
            if hasattr(obj, "state_dict"):
                return obj.state_dict()
            if isinstance(obj, dict):
                return obj
    return ckpt  # evtl. bereits state_dict

sd12_det  = load_ckpt("yolo12n.pt")        # v12 DET
sd11_seg  = load_ckpt("yolo11n-seg.pt")    # v11 SEG

def copy_matching(src, dst, name_filter=lambda k: True, tag=""):
    copied = skipped = 0
    for k, v in src.items():
        if not name_filter(k):
            continue
        if k in dst and dst[k].shape == v.shape:
            dst[k] = v.clone()
            copied += 1
        else:
            skipped += 1
    print(f"[{tag}] copied={copied}, skipped={skipped}")
    return copied

# 1) v12 Detection → v12 Seg (Backbone/Neck etc.)
copy_matching(sd12_det, sd_target, tag="v12-det -> v12-seg")

# 2) v11 Seg → v12 Seg (nur offensichtliche Mask-Keys; vieles wird nicht passen)
seg_hints = ("mask", "seg", "proto")
copy_matching(sd11_seg, sd_target,
              name_filter=lambda k: any(h in k.lower() for h in seg_hints),
              tag="v11-seg -> v12-seg (head)")

# 3) ins Modell laden und speichern
m.model.load_state_dict(sd_target, strict=False)
m.save("y12n_seg_hybrid.pt")
print("Hybrid gespeichert: y12n_seg_hybrid.pt")

[v12-det -> v12-seg] copied=691, skipped=8
[v11-seg -> v12-seg (head)] copied=0, skipped=20
Hybrid gespeichert: y12n_seg_hybrid.pt


In [ ]:
import torch
from ultralytics import YOLO

# Zielarchitektur: v12 SEG, frisch initialisiert
m = YOLO("yolo12s-seg.yaml")
sd_target = m.model.state_dict()

def load_ckpt(path):
    # Fix für PyTorch 2.6: explizit weights_only=False
    ckpt = torch.load(path, map_location="cpu", weights_only=False)
    # typische Ultralytics-Container auflösen
    for k in ("model", "ema", "state_dict"):
        if k in ckpt:
            obj = ckpt[k]
            if hasattr(obj, "state_dict"):
                return obj.state_dict()
            if isinstance(obj, dict):
                return obj
    return ckpt  # evtl. bereits state_dict

sd12_det  = load_ckpt("yolo12s.pt")        # v12 DET
sd11_seg  = load_ckpt("yolo11s-seg.pt")    # v11 SEG

def copy_matching(src, dst, name_filter=lambda k: True, tag=""):
    copied = skipped = 0
    for k, v in src.items():
        if not name_filter(k):
            continue
        if k in dst and dst[k].shape == v.shape:
            dst[k] = v.clone()
            copied += 1
        else:
            skipped += 1
    print(f"[{tag}] copied={copied}, skipped={skipped}")
    return copied

# 1) v12 Detection → v12 Seg (Backbone/Neck etc.)
copy_matching(sd12_det, sd_target, tag="v12-det -> v12-seg")

# 2) v11 Seg → v12 Seg (nur offensichtliche Mask-Keys; vieles wird nicht passen)
seg_hints = ("mask", "seg", "proto")
copy_matching(sd11_seg, sd_target,
              name_filter=lambda k: any(h in k.lower() for h in seg_hints),
              tag="v11-seg -> v12-seg (head)")

# 3) ins Modell laden und speichern
m.model.load_state_dict(sd_target, strict=False)
m.save("y12s_seg_hybrid.pt")
print("Hybrid gespeichert: y12s_seg_hybrid.pt")

[v12-det -> v12-seg] copied=691, skipped=8
[v11-seg -> v12-seg (head)] copied=0, skipped=20
Hybrid gespeichert: y12s_seg_hybrid.pt


In [ ]:
from ultralytics import YOLO
from datetime import datetime
from google.colab import runtime

data_yaml = "/content/drive/MyDrive/RoboCup_Trainingsdatensatz/data.yaml"
common = dict(
    task="segment",
    data=data_yaml,
    imgsz=832, batch=32, workers=4,
    epochs=300, patience=60,
    cache=True, deterministic=True, seed=0,
    save=True, save_period=25,
    # Augmentations:
    fliplr=0.5, flipud=0.10, degrees=10.0, shear=3.0, perspective=0.000,
    scale=0.50, translate=0.20,
    hsv_h=0.02, hsv_s=0.80, hsv_v=0.50,
    mosaic=0.8, close_mosaic=10, mixup=0.10, copy_paste=0.20,
    erasing=0.50
)

# 1) YOLOv12n-seg from scratch
run_n = f"y12nseg_scratch_832_{datetime.now().strftime('%Y%m%d_%H%M')}"
model_n = YOLO("y12n_seg_hybrid.pt")
model_n.train(project="/content/drive/MyDrive/YOLO_Runs/segment", name=run_n, **common)

runtime.unassign()


Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/RoboCup_Trainingsdatensatz/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.5, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=y12n_seg_hybrid.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=y12nseg_scratch_832_20251003_2047, nbs=64, nms=False, opset=None, optimize=False, opti

In [ ]:
from ultralytics import YOLO
from datetime import datetime
from google.colab import runtime

data_yaml = "/content/drive/MyDrive/RoboCup_Trainingsdatensatz/data.yaml"
common = dict(
    task="segment",
    data=data_yaml,
    imgsz=832, batch=32, workers=4,
    epochs=300, patience=60,
    cache=True, deterministic=True, seed=0,
    save=True, save_period=25,
    # Augmentations:
    fliplr=0.5, flipud=0.10, degrees=10.0, shear=3.0, perspective=0.000,
    scale=0.50, translate=0.20,
    hsv_h=0.02, hsv_s=0.80, hsv_v=0.50,
    mosaic=0.8, close_mosaic=10, mixup=0.10, copy_paste=0.20,
    erasing=0.50
)

# 1) YOLOv12n-seg from scratch
run_n = f"y12sseg_scratch_832_{datetime.now().strftime('%Y%m%d_%H%M')}"
model_n = YOLO("y12s_seg_hybrid.pt")
model_n.train(project="/content/drive/MyDrive/YOLO_Runs/segment", name=run_n, **common)

runtime.unassign()

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/RoboCup_Trainingsdatensatz/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.5, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=y12s_seg_hybrid.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=y12sseg_scratch_832_20251004_1104, nbs=64, nms=False, opset=None, optimize=False, opti

## Convertion for further Purposes(FAILED: YOLO12 not compatible to OpenVINO)

In [ ]:
%pip uninstall -y numpy
%pip install -U "numpy<2.0" "luxonis-ml[data]" modelconv==0.4.0
%pip install -U depthai==3.0.0rc2 depthai-nodes==0.3.0
# Runtime hart neu starten, damit die neuen Wheels geladen werden:
import os, sys; os.kill(os.getpid(), 9)

In [ ]:
import shutil

shutil.copy("/content/drive/MyDrive/YOLO_Runs/segment/y11nseg_640_b32_pat50_20250929_2106/weights/best.pt", "yolo12n_instance_segmentation_trained.pt")

'yolo11n_instance_segmentation_trained.pt'

In [ ]:
HUBAI_API_KEY = "tapi.S4zbE9uV-5yAedJy2F2ZAQ.tdniU5AkznsZRJvCt_jEwcLCpERpgU7dpDqflqyXv1X6Q9-Bm9vGe8eUTe7zWtGC7M2zVJTPYQXs1mshhPtdEw"

In [ ]:
from modelconverter import convert

converted_model = convert.RVC2(
    api_key=HUBAI_API_KEY,
    path="/content/yolo12n_instance_segmentation_trained.pt",
    name="YOLO12 Nano RoboCup Instance Segmentation",
    description_short="x",
    yolo_version="yolov12",
    yolo_input_shape="832 832",
    yolo_class_names=["Kugellager", "Motor", "Schraubenzieher"],
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False
)